In [2]:
from agents import Agent, Runner, function_tool, ItemHelpers


@function_tool
def get_weather(city: str):
    """Get weather by city"""
    return "30 degrees"


agent = Agent(
    name="Assistant Agent",
    instructions="You are a helpful assistant. Use tools when needed to answer questions",
    tools=[get_weather],
)

stream = Runner.run_streamed(
    agent, "Hello how are you? What is the weather in the capital of Spain?"
)

async for event in stream.stream_events():
    if event.type == "raw_response_event":
        continue
    elif event.type == "agent_updated_stream_event":
        print("Agent updated to", event.new_agent.name)
    elif event.type == "run_item_stream_event":
        if event.item.type == "tool_call_item":
            print(event.item.raw_item.to_dict())
        elif event.item.type == "tool_call_output_item":
            print(event.item.output)
        elif event.item.type == "message_output_item":
            print(ItemHelpers.text_message_output(event.item))
    print("=" * 20)

Agent updated to Assistant Agent
{'arguments': '{"city":"Madrid"}', 'call_id': 'call_uIG4sin48WQdktLrm1Ntaz1X', 'name': 'get_weather', 'type': 'function_call', 'id': 'fc_03f0f5d44b2923ed00692564fe02dc819dbca977ff25b2e5c1', 'status': 'completed'}
30 degrees
Hello! I'm doing well, thank you for asking. The current weather in Madrid, the capital of Spain, is 30 degrees. If you need more details or updates, let me know!


In [5]:
from agents import Agent, Runner, function_tool, ItemHelpers


@function_tool
def get_weather(city: str):
    """Get weather by city"""
    return "30 degrees"


agent = Agent(
    name="Assistant Agent",
    instructions="You are a helpful assistant. Use tools when needed to answer questions",
    tools=[get_weather],
)

stream = Runner.run_streamed(
    agent, "Hello how are you? What is the weather in the capital of Spain?"
)

message = ""
args = ""

async for event in stream.stream_events():
    if event.type == "raw_response_event":
        event_type = event.data.type
        if event_type == "response.output_text.delta":
            message += event.data.delta
            print(message)
        elif event_type == "response.function_call_arguments.delta":
            args += event.data.delta
            print(args)
        elif event_type == "response.completed":
            message = ""
            args = ""

{"
{"city
{"city":"
{"city":"Madrid
{"city":"Madrid"}
Hello
Hello!
Hello! I'm
Hello! I'm here
Hello! I'm here to
Hello! I'm here to help
Hello! I'm here to help you
Hello! I'm here to help you.
Hello! I'm here to help you. The
Hello! I'm here to help you. The weather
Hello! I'm here to help you. The weather in
Hello! I'm here to help you. The weather in Madrid
Hello! I'm here to help you. The weather in Madrid,
Hello! I'm here to help you. The weather in Madrid, the
Hello! I'm here to help you. The weather in Madrid, the capital
Hello! I'm here to help you. The weather in Madrid, the capital of
Hello! I'm here to help you. The weather in Madrid, the capital of Spain
Hello! I'm here to help you. The weather in Madrid, the capital of Spain,
Hello! I'm here to help you. The weather in Madrid, the capital of Spain, is
Hello! I'm here to help you. The weather in Madrid, the capital of Spain, is currently
Hello! I'm here to help you. The weather in Madrid, the capital of Spain, is currently 